# ICIJ Offshore leaks dataset

In [1]:
from dtgraph import Neo4jGraph, Rule, Transformation
hostname = "localhost"
uri = f"bolt://{hostname}:7687"
graph = Neo4jGraph(uri, database="neo4j", username="", password="")

In [2]:
panama_address_to_movies = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
GENERATE 
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_code):T_Country {
    name = a.country
})
''')

In [3]:
my_transform = Transformation([panama_address_to_movies], with_diagnose = False)
for _ in range(5):
    my_transform.apply_on(graph)
    my_transform.abort()

Index: Added 0 index, completed after 1 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3141 ms.
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
MERGE (x:_dummy {
    _id: "(" + elementID(a) + ")" 
})
ON CREATE
    SET x:T_Address,
        x.source = a.sourceID
ON MATCH
    SET x:T_Address,
        x.source = 
        CASE
            WHEN x.source <> a.sourceID THEN
                "Conflict Detected!"
            ELSE
                a.sourceID
        END
MERGE (y:_dummy {
    _id: "(" + a.country_code + ")" 
})
ON CREATE
    SET y:T_Country,
        y.name = a.country
ON MATCH
    SET y:T_Country,
        y.name = 
        CASE
            WHEN y.name <> a.country THEN
                "Conflict Detected!"
            ELSE
                a.country
        END
MERGE (x)-[x_2:T_LOCATED {
    _id: "(" + ":T_LOCATED:" + "," + elementID(x) + "," + elem

Index: Removed 1 index, completed after 2 ms.
Abort: Deleted 123433 nodes, deleted 123240 relationships, completed after 781 ms.
